In [ ]:
import os
import shutil
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms

from pathlib import Path

from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST

from google.cloud import storage

In [ ]:
# When using Colab, you need to allow it to access your resources on GCP.
try:
    from google.colab import auth
    auth.authenticate_user()
except:
    pass

# Fill in the project-id of your GCP project and the name of 
# the GCS bucket where you want to store the checkpoints
PROJECT = "my_gcp_project"
BUCKET = "my_gcp_bucket"

# Checkpoint helper functions

This section contains the helper functions for storing and loading Tensorflow/Keras checkpoints on Google Cloud Storage. You can use these functions as a starting point and modify them for your use case.

The only dependency for this boilerplate is the Python Client for Google Cloud Storage, you can find the docs here:

https://googleapis.dev/python/storage/latest/index.html

## Helper functions for local file system manipulation

Helper functions for local file system manipulation. Tensorflow does not support to save a checkpoint in an archived format such as `zip` or `tar`. Archiving checkpoints before sending them to GCS can help reducing necessary storage and make the file structure on gcs simpler. It is not necessary to archive checkpoints.

In [ ]:
def list_files(directory: str, recursive: bool = True):
    """Create a generator that lists all files in a directory.
    Optionally also recusrively all list files in subdirectories.

    Args:
        directory (str): List files in this directory.
        recursive (bool, optional): Option to recusrively list files
        in subdirectories. Defaults to True.

    Returns:
        Generator[str]: generator of filepaths in string format.
    """
    if recursive:
        filepaths = Path(directory).rglob('*')
    else:
        filepaths = Path(directory).glob('*')

    return (str(child) for child in filepaths if child.is_file())


def clear_directory(directory: str):
    """Removes all files and subdirectories in a given directory.

    Args:
        directory (str): Path to the directory that will be cleared.
    """
    shutil.rmtree(directory)
    os.mkdir(directory)


def zip_directory(directory: str,
                  output_file_path: str,
                  archive_format: str = 'zip'):
    """Archives a directory with all its files and subdirectories

    Args:
        directory (str): Directory that will be archived.
        output_file_path (str): Path where the checkpoint will be saved.
        archive_format (str, optional): Type of archival. Defaults to 'zip'.
    """
    # Extract the filepath without the file extension,
    # (shutil automatically adds the extension)
    file_path = os.path.splitext(output_file_path)[0]
    shutil.make_archive(file_path, archive_format, directory)

## Helper functions for moving files/directories on GCS

Helper functions move a file or directory on GCS so it can be new file or directory can take its place without losing the previous one. This is useful to save the latest version of a file or directory in a fixed location.

In [ ]:
def move_gcs_file(old_file_path: str, new_file_path: str, bucket):
    """Move a file on gcs to a new location.

    Args:
        old_file_path (str): Path to the original file location.
        new_file_path (str): Path to the new location of the file.
        bucket (bucket): GCS bucket that contains the file.
    """
    blob = bucket.blob(old_file_path)
    if blob.exists():
        bucket.rename_blob(blob, new_file_path)


def move_gcs_directory(old_directory_path: str,
                       new_directory_path: str,
                       bucket):
    """Move a directory on gcs to a new location

    Args:
        old_directory_path (str): Path to the original directory location.
        new_directory_path (str): Path to the new location of the directory.
        bucket (bucket): GCS bucket that contains the directory.
    """
    blobs = bucket.list_blobs(prefix=old_directory_path)
    for blob in blobs:
        file_path = blob.name.replace(old_directory_path, new_directory_path)
        bucket.rename_blob(blob, file_path)

## Helper functions to download files/directories from GCS

Helper functions to download files, directories and archived files from GCS to the local file system, these can be used to restore a checkpoint from GCS.

In [ ]:
def download_gcs_file(gcs_file_path: str,
                      local_file_path: str,
                      bucket: storage.bucket.Bucket):
    """Download a file from gcs to a given location.

    Args:
        gcs_file_path (str): Path to the file on GCS.
        local_file_path (str): Local path to save the file.
        bucket (bucket): GCS bucket where the file is stored.
    """
    blob = bucket.blob(gcs_file_path)
    blob.download_to_filename(local_file_path)


def download_gcs_directory(gcs_directory_path: str,
                           local_directory_path: str,
                           bucket: storage.bucket.Bucket):
    """Download a directory with all its subdirectories from GCS.
    (The directory structure is recreated locally)

    Args:
        gcs_directory_path (str): Path to the directory on GCS.
        local_directory_path (str): Local path where the
        directory will be stored.
        bucket (bucket): GCS bucket where the directory is stored.
    """
    blobs = bucket.list_blobs(prefix=gcs_directory_path)

    for blob in blobs:
        local_file_path = blob.name.replace(
            gcs_directory_path,
            local_directory_path
        )
        # Recreate the directory structure if necessary.
        path = Path(local_file_path).parents[0]
        path.mkdir(parents=True, exist_ok=True)
        blob.download_to_filename(local_file_path)


def download_gcs_archive(gcs_file_path: str,
                         local_file_path: str,
                         unpack_directory: str,
                         bucket: storage.bucket.Bucket):
    """Downloads an archived file from GCS and unpacks it.

    Args:
        gcs_file_path (str): Path to the archived file on GCS.
        local_file_path (str): Local path to save the archived file.
        unpack_directory (str): Local directory where the
        archived file will be unpacked.
        bucket (bucket): GCS bucket where the archive is stored.
    """
    download_gcs_file(gcs_file_path, local_file_path, bucket)
    shutil.unpack_archive(local_file_path, unpack_directory)

## Helper functions to send files/directories to GCS

Helper functions to store local files, directories and archived files on GCS, these can be used to backup a checkpoint.

In [ ]:
def write_file_to_gcs(local_file_path: str,
                      gcs_file_path: str,
                      bucket: storage.bucket.Bucket):
    """Writes a single file to gcs. The effect of uploading to
    an existing blob depends on the “versioning” and “lifecycle”
    policies defined on the blob’s bucket. In the absence of
    those policies, upload will overwrite any existing contents.
    (https://googleapis.dev/python/storage/latest/blobs.html)

    Args:
        local_file_path (str): Path to the local file that will be sent to GCS.
        gcs_file_path (str): Location on gcs where the file will be stored.
        bucket (bucket): GCS bucket where the file will be stored.
    """
    blob = bucket.blob(gcs_file_path)
    blob.upload_from_filename(Path(local_file_path).absolute())


def write_directory_to_gcs(local_directory_path: str,
                           gcs_directory_path: str,
                           bucket: storage.bucket.Bucket,
                           recursive: bool = True):
    """Write a directory (optionally all its subdirectories) to gcs.
    The effect of uploading to an existing blob depends on the
    “versioning” and “lifecycle” policies defined on the blob’s bucket. In
    the absence of those policies, upload will overwrite any existing contents.
    (https://googleapis.dev/python/storage/latest/blobs.html)

    Args:
        local_directory_path (str): Local path to the directory
        that will be sent to GCS.
        gcs_directory_path (str): Path on the GCS bucket where
        the directory will be stored.
        bucket (bucket): GCS bucket where the directory will be stored.
        recursive (bool, optional): Recursively search
        subdirecties. Defaults to True.
    """
    for local_file_path in list_files(local_directory_path, recursive):
        gcs_file_path = local_file_path.replace(
            local_directory_path,
            gcs_directory_path
        )
        blob = bucket.blob(gcs_file_path)
        blob.upload_from_filename(local_file_path)

### Pytorch example function to save a checkpoint

In [ ]:
def save_pt_checkpoint(model_state: dict,
                       local_directory: str,
                       local_checkpoint: str,
                       gcs_latest_checkpoint: str,
                       gcs_checkpoint_store: str,
                       bucket: storage.bucket.Bucket):
    """Send a checkpoint in archived form to GCS,
    moves the current latest checkpoint to a different,
    specified location.

    Args:
        model_state (dict): Dictionary that contains all
        checkpoint data that needs to be saved.
        local_directory (str): Local path to the directory
        where the checkpoint will be saved.
        local_checkpoint (str): Local path where
        the checkpoint will be saved.
        gcs_latest_checkpoint (str): Path where the latest
        checkpoint is saved on GCS.
        gcs_checkpoint_store (str): Path where the current
        last checkpoint on GCS will be moved to.
        bucket (bucket): GCS bucket where the checkpoint will be stored.
    """
    # clear the directory where the checkpoint is saved locally
    clear_directory(local_directory)
    move_gcs_file(gcs_latest_checkpoint, gcs_checkpoint_store, bucket)
    torch.save(model_state, local_checkpoint)
    write_file_to_gcs(local_checkpoint, gcs_latest_checkpoint, bucket)

### Pytorch example function to load a checkpoint

In [ ]:
def load_pt_checkpoint(gcs_checkpoint: str,
                       local_checkpoint: str,
                       bucket: storage.bucket.Bucket):
    """Load an archived checkpoint from GCS

    Args:
        gcs_checkpoint (str): Path to the checkpoint on GCS.
        local_checkpoint (str): Local path to the checkpoint.
        bucket (bucket): GCS bucket that contains the checkpoint
    """
    download_gcs_file(gcs_checkpoint, local_checkpoint, bucket)
    checkpoint = torch.load(local_checkpoint)
    return checkpoint

# Training example

In [ ]:
# flatten 28*28 images to a 784 vector for each image
transform = transforms.Compose([
    transforms.ToTensor(),  # convert to tensor
    transforms.Lambda(lambda x: x.view(-1))  # flatten into vector
])

trainset = MNIST(".", train=True, download=True, transform=transform)
testset = MNIST(".", train=False, download=True, transform=transform)

# create data loaders
trainloader = DataLoader(trainset, batch_size=128, shuffle=True)
testloader = DataLoader(testset, batch_size=128, shuffle=True)

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = F.relu(out)
        out = self.fc2(out)
        if not self.training:
            out = F.softmax(out, dim=1)
        return out

In [ ]:
storage_client = storage.Client(project=PROJECT)
bucket = storage_client.get_bucket(BUCKET)
local_directory = "checkpoint-buffer"
gcs_directory = "pt-mnist-checkpoints"
checkpoint_name = "latest-checkpoint.pt"
local_checkpoint = f"{local_directory}/{checkpoint_name}"
gcs_latest_checkpoint = f"{gcs_directory}/{checkpoint_name}"

#os.mkdir(local_directory)

In [ ]:
model = MLP(784, 784, 10)

# define the loss function and the optimiser
loss_function = nn.CrossEntropyLoss()
optimiser = optim.Adam(model.parameters())

# Load the checkpoint from GCS.
checkpoint = load_pt_checkpoint(
    gcs_latest_checkpoint,
    local_checkpoint, bucket
)
# Read the checkpoint values and prepare for training.
model.load_state_dict(checkpoint['model_state_dict'])
optimiser.load_state_dict(checkpoint['optimiser_state_dict'])
start_epoch = checkpoint['epoch']


# the epoch loop
for epoch in range(start_epoch, 5):
    
    running_loss = 0.0

    for data in trainloader:
        # get the inputs
        inputs, labels = data

        # zero the parameter gradients
        optimiser.zero_grad()

        # forward + loss + backward + optimise (update weights)
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimiser.step()

        # keep track of the loss this epoch
        running_loss += loss.item()

    # Data to save in the checkpoint.
    checkpoint_data = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimiser_state_dict': optimiser.state_dict(),
        'loss': running_loss,
    }

    # Save the model to GCS.
    save_pt_checkpoint(
        checkpoint_data,
        local_directory,
        local_checkpoint,
        gcs_latest_checkpoint,
        f"{gcs_directory}/checkpoint-{epoch}.pt",
        bucket
    )